# Ion-Site Occupancy Recommender System Demonstration

This notebook guides you through the process of obtaining new ion-site occupancies based on a given OQMD ID that represents the structural prototype of interest.

As an example, we will delve into the crystal structure prototype of [$\text{CsV}_3\text{Sb}_5$](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.125.247002), a compound that recently piqued interest for its novel properties. The visualization of this unique crystal structure is displayed below:

![kagome_image](images/kagome_structure.png "Crystal Structure of CsV3Sb5")

Corresponding to this structure, the OQMD entry with the ID: 1514955 can be found at this [link](https://oqmd.org/materials/entry/1514955). Our aim in this notebook is to demonstrate the process of recommending new ion-site occupancies using this structural prototype as a reference.


The classes used to build the Recommender System are defined in `recommender/core.py` file.

In [1]:
from recommender.core import RecommenderSystem, OccupationData
import joblib
from gensim.models import Word2Vec
from pymatgen.core import Element

### Loading embedding and defining the data location

In this section, we establish the path to the occupancy data that was used to construct the NetworkX graph. This graph can be found in `data/G100.gexf`. Simultaneously, we load the Word2Vec model, which includes the ion and site embeddings.

We also load the Decision Tree classifier to determine the distance threshold applied within the embedding space. 

In [2]:
json_file_path = 'data/occupancy_data.json.gz'

model = Word2Vec.load(f'models/word2vec.model')
embedding = model.wv

clf = joblib.load(f'models/decision_tree.joblib')
distance_threshold = clf.tree_.threshold[0]

Here we are defining ions that we don't desire to be recommended (noble gases and radioctive elements)

In [3]:
ions = ['Xe', 'Kr', 'Ar', 'Ne', 'He', 'I', 'Br', 'Cl', 'F', 'Te', 'Se', 'S', 'O', 'Bi', 'Sb', 'As', 'P', 'N', 'Pb', 'Sn', 'Ge', 'Si', 'C', 'Tl', 'In', 'Ga', 'Al', 'B', 'Hg', 'Cd', 'Zn', 'Au', 'Ag', 'Cu', 'Pt', 'Pd', 'Ni', 'Ir', 'Rh', 'Co', 'Os', 'Ru', 'Fe', 'Re', 'Tc', 'Mn', 'W', 'Mo', 'Cr', 'Ta', 'Nb', 'V', 'Hf', 'Zr', 'Ti', 'Pu', 'Np', 'U', 'Pa', 'Th', 'Ac', 'Lu', 'Yb', 'Tm', 'Er', 'Ho', 'Dy', 'Tb', 'Gd', 'Eu', 'Sm', 'Pm', 'Nd', 'Pr', 'Ce', 'La', 'Y', 'Sc', 'Ba', 'Sr', 'Ca', 'Mg', 'Be', 'Cs', 'Rb', 'K', 'Na', 'Li', 'H']
ion_forbidden_list = [ion for ion in ions if (Element(ion).Z > 83 or 
                                              ion in ['Tc', 'Pm'] or
                                              Element(ion).group == 18)]
ion_forbidden_list 

['Xe', 'Kr', 'Ar', 'Ne', 'He', 'Tc', 'Pu', 'Np', 'U', 'Pa', 'Th', 'Ac', 'Pm']

First, we create a `OccupationData` object with the compressed json file path.

In [4]:
occupation_data = OccupationData(json_file_path)

At this stage, we retrieve an `AnonymousMotif` object for a specific OQMD ID. This object encapsulates all the crystal structure data, as defined by the Anonymous Motif concept outlined in the paper. This structure-centric information is the foundation for generating our ion-site occupancy recommendations.

In [14]:
kagome_id = 1514955
kagome_AM = occupation_data.get_AM_from_OQMD_id(kagome_id)

The `AnonymousMotif` object provides a prototype Pymatgen Structure consisting exclusively of carbon (C) atoms. It also comes with its equivalent site indexes. After receiving the occupancy recommendations, these resources can be utilized for executing ionic substitutions, effectively transforming the prototype into new material structures.

In [15]:
kagome_AM.example_structure

Structure Summary
Lattice
    abc : 5.469073129 5.469073129363581 9.384302298
 angles : 90.0 90.0 120.00000000384938
 volume : 243.08607542522887
      A : 5.469073129 0.0 0.0
      B : -2.734536565 4.736356265 0.0
      C : 0.0 0.0 9.384302298
    pbc : True True True
PeriodicSite: C (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: C (2.7345, 1.5788, 2.4107) [0.6667, 0.3333, 0.2569]
PeriodicSite: C (-0.0000, 3.1576, 2.4107) [0.3333, 0.6667, 0.2569]
PeriodicSite: C (0.0000, 0.0000, 4.6922) [0.0000, 0.0000, 0.5000]
PeriodicSite: C (2.7345, 1.5788, 6.9736) [0.6667, 0.3333, 0.7431]
PeriodicSite: C (-0.0000, 3.1576, 6.9736) [0.3333, 0.6667, 0.7431]
PeriodicSite: C (2.7345, 0.0000, 4.6922) [0.5000, 0.0000, 0.5000]
PeriodicSite: C (-1.3673, 2.3682, 4.6922) [0.0000, 0.5000, 0.5000]
PeriodicSite: C (1.3673, 2.3682, 4.6922) [0.5000, 0.5000, 0.5000]

In [16]:
kagome_AM.equivalent_sites_indexes

[0, 1, 1, 3, 1, 1, 6, 6, 6]

Next, we'll instantiate a `RecommenderSystem` object. This process involves integrating several elements including `OccupationData`, node embeddings, a predefined distance threshold (given by the trained decision tree), and a list of ions that are deemed unsuitable for inclusion (`ion_forbidden_list`).

One can further refine the model's recommendations by setting a smaller distance threshold, which makes the recommendation process more stringent.

In [12]:
rs = RecommenderSystem(occupation_data=occupation_data, 
                       embedding=embedding, 
                       distance_threshold=distance_threshold, 
                       ion_forbidden_list=ion_forbidden_list)

To generate recommendations for this prototype structure, we feed the `AnonymousMotif`—derived from the OQMD ID—into the `get_recommendations_for_AM` method of the `RecommenderSystem`. The output is a dictionary where each key corresponds to an `AMsite` object representing a specific Anonymous Motif site. Each key is associated with a value that comprises a list of recommended ions for that specific site.

In [18]:
kagome_recommendations = rs.get_recommendation_for_AM(kagome_AM)
kagome_recommendations

{(191, 9, (1, '6/mmm'), 0), site index: 0: [('Rb',
   0.018323421478271484,
   False),
  ('K', 0.02188342809677124, False),
  ('Cs', 0.02744007110595703, False),
  ('Tl', 0.11811000108718872, False),
  ('Na', 0.13025516271591187, False),
  ('Ba', 0.261111319065094, True)],
 (191, 9, (1, '6/mmm'), 0), site index: 3: [('Sb',
   0.003526031970977783,
   False),
  ('Bi', 0.028451979160308838, False),
  ('As', 0.07530736923217773, False),
  ('P', 0.15748435258865356, False),
  ('Ir', 0.24137479066848755, True),
  ('Pt', 0.28948670625686646, True),
  ('Ta', 0.2916148900985718, True),
  ('Re', 0.3077079653739929, True),
  ('Rh', 0.3191390633583069, True),
  ('Au', 0.3234304189682007, True),
  ('Os', 0.3247630000114441, True)],
 (191, 9, (3, 'mmm'), 0), site index: 6: [('V', 0.03470265865325928, False),
  ('Nb', 0.07285183668136597, False),
  ('Ta', 0.08547395467758179, False),
  ('Mn', 0.08579069375991821, False),
  ('Ti', 0.09940123558044434, True),
  ('Cr', 0.10330325365066528, False),
  ('

The recommendations are expressed as tuples in the format `(ion, ion-site distance, novel occupation)`. Here, `novel occupation` indicates whether the suggested ion-site occupation represents a new variant within the OQMD compound set (the dataset used to construct the recommender system). The recommendations are sorted based on the ion-site distance.

Importantly, the `AMSite` keys display their respective site indexes. These indexes reference specific sites in the `AnonymousMotif.example_structure` where the recommended ions should be substituted, thus providing a clear mapping for potential new compounds.

In [9]:
rs.get_recommendation_for_site('221_5_1(m-3m)_1(m-3m)_3(4/mmm)_0[2:3(4/mmm)]')

[('F', 0.0973658561706543, False),
 ('Cl', 0.10440915822982788, False),
 ('Br', 0.12838470935821533, False),
 ('H', 0.16112929582595825, False),
 ('O', 0.17956966161727905, False),
 ('I', 0.2550504207611084, False)]

In [19]:
rs.get_recommendation_for_ion('W')

[('15_24_2(2)_2(2)_4(1)_4(1)_4(1)_4(1)_4(1)_1[6:4(1)]',
  0.0004812479019165039,
  False),
 ('11_14_2(-1)_2(m)_2(m)_4(1)_4(1)_0[2:2(m)]', 0.0005944967269897461, False),
 ('111_7_1(-42m)_2(222)_4(m)_1[0:1(-42m)]', 0.0006749629974365234, False),
 ('13_24_2(2)_2(2)_4(1)_4(1)_4(1)_4(1)_4(1)_0[6:4(1)]',
  0.0007376670837402344,
  False),
 ('2_13_1(-1)_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2[6:2(1)]',
  0.0008211731910705566,
  False),
 ('2_12_1(-1)_1(-1)_2(1)_2(1)_2(1)_2(1)_2(1)_0[6:2(1)]',
  0.0010685920715332031,
  False),
 ('8_7_1(m)_1(m)_1(m)_1(m)_1(m)_1(m)_1(m)_1[5:1(m)]',
  0.0012214183807373047,
  False),
 ('2_13_1(-1)_2(1)_2(1)_2(1)_2(1)_2(1)_2(1)_2[1:2(1)]',
  0.001889944076538086,
  False),
 ('63_10_2(m2m)_2(m2m)_2(m2m)_4(m)_1[1:2(m2m)]', 0.0021080374717712402, False),
 ('12_10_2(m)_2(m)_2(m)_2(m)_2(m)_16[3:2(m)]', 0.002344489097595215, False),
 ('12_10_2(m)_2(m)_2(m)_2(m)_2(m)_16[4:2(m)]', 0.0023616552352905273, False),
 ('8_7_1(m)_1(m)_1(m)_1(m)_1(m)_1(m)_1(m)_1[6:1(m)]',
  0.0024210810